In [ ]:
#get the notebook, env ready
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
#read the data
df=pd.read_csv('college.csv',index_col=0)
df.describe(include='all')

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
count,777,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.00000
unique,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,3001.638353,2018.804376,779.972973,27.558559,55.796654,3699.907336,855.298584,10440.669241,4357.526384,549.380952,1340.642214,72.660232,79.702703,14.089704,22.743887,9660.171171,65.46332
std,NaN,3870.201484,2451.113971,929.176190,17.640364,19.804778,4850.420531,1522.431887,4023.016484,1096.696416,165.105360,677.071454,16.328155,14.722359,3.958349,12.391801,5221.768440,17.17771
min,NaN,81.000000,72.000000,35.000000,1.000000,9.000000,139.000000,1.000000,2340.000000,1780.000000,96.000000,250.000000,8.000000,24.000000,2.500000,0.000000,3186.000000,10.00000
25%,NaN,776.000000,604.000000,242.000000,15.000000,41.000000,992.000000,95.000000,7320.000000,3597.000000,470.000000,850.000000,62.000000,71.000000,11.500000,13.000000,6751.000000,53.00000
50%,NaN,1558.000000,1110.000000,434.000000,23.000000,54.000000,1707.000000,353.000000,9990.000000,4200.000000,500.000000,1200.000000,75.000000,82.000000,13.600000,21.000000,8377.000000,65.00000
75%,NaN,3624.000000,2424.000000,902.000000,35.000000,69.000000,4005.000000,967.000000,12925.000000,5050.000000,600.000000,1700.000000,85.000000,92.000000,16.500000,31.000000,10830.000000,78.00000


In [ ]:
#grad rate cannot be higher than 100 so we just clean up and max at 100
df=df[df['Grad.Rate']<=100]

In [ ]:
y=df['Private']
X=df.drop(['Private'],axis=1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
rf = RandomForestClassifier(n_estimators=50, random_state=0, class_weight='balanced')
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=50, random_state=0)

In [ ]:
#first is accuracy, then is AUC score
print(rf.score(X_val, y_val))
print(roc_auc_score(y_val, rf.predict_proba(X_val)[:,1]))

0.927038626609442
0.9773109243697479


In [ ]:
#set the parameters. If "max features" is = to total, bootsrap. If n-1, random sample without replace.
param = {
    'n_estimators':[50,100],
    'max_features':[1,18]
}

In [ ]:
#create 2nd random forest
#similar to first forest, except that this uses the above parameters.
#creating a total of 34 random forest algorithms. One with 50 estimators, one with 100.
#Both with 17 each, so 34
#This is bagging since the sample variables is being used in all the trees
rf2=RandomForestClassifier(random_state=0, class_weight='balanced')
grid=GridSearchCV(estimator=rf2, param_grid=param, n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=0),
             n_jobs=-1,
             param_grid={'max_features': [1, 18], 'n_estimators': [50, 100]})

In [ ]:
#check the best parameter
#based on average performance of auto bag sample, 50 trees with each 1 feature.
grid.best_params_

{'max_features': 1, 'n_estimators': 50}

In [ ]:
print(grid.score(X_val, y_val))
print(roc_auc_score(y_val, grid.predict_proba(X_val)[:,1]))

0.9098712446351931
0.9731092436974791


In [ ]:
final_model=grid.best_estimator_

In [ ]:
feature_importances = pd.DataFrame(final_model.feature_importances_,
                                   index = X_train.columns, columns=['importance']).sort_values('importance',ascending=False)
display(feature_importances)

,importance
Enroll,0.116129
Outstate,0.109878
F.Undergrad,0.105072
Accept,0.079118
P.Undergrad,0.077881
Room.Board,0.061203
S.F.Ratio,0.059768
Expend,0.055433
Grad.Rate,0.050795
perc.alumni,0.048885
